In [ ]:
from flask import Flask, request, jsonify
import pyodbc
import joblib  # Hoặc pickle cho model
import pandas as pd

app = Flask(__name__)

# Kết nối Azure SQL Server
server = 'your_server.database.windows.net'
database = 'your_database'
username = 'your_username'
password = 'your_password'
driver = '{ODBC Driver 18 for SQL Server}'

conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}')

# Load mô hình ML
model = joblib.load('path_to_your_model.pkl')

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    user_input = data.get('user_input', '')
    location = data.get('location', '')
    categories = data.get('categories', [])

    if not user_input or not location:
        return jsonify({"status": "error", "message": "Missing required fields"}), 400

    # Query dữ liệu từ SQL Server
    query = f"""
    SELECT business_id, name, categories, stars, review_count, address
    FROM business_data
    WHERE location LIKE ? AND categories LIKE ?
    """
    categories_filter = f"%{'%'.join(categories)}%" if categories else "%"
    cursor = conn.cursor()
    cursor.execute(query, (f"%{location}%", categories_filter))
    businesses = cursor.fetchall()

    # Tạo DataFrame để xử lý ML
    df = pd.DataFrame.from_records(
        businesses,
        columns=["business_id", "name", "categories", "stars", "review_count", "address"]
    )

    # Dự đoán với mô hình
    df['predicted_score'] = model.predict_proba(df['name'] + " " + user_input)[:, 1]

    # Sắp xếp và chọn top kết quả
    suggestions = df.sort_values('predicted_score', ascending=False).head(5)
    results = suggestions.to_dict(orient='records')

    # Output API
    return jsonify({
        "status": "success",
        "input": user_input,
        "suggestions": results
    })

if __name__ == '__main__':
    app.run(debug=True)
